<a href="https://colab.research.google.com/github/brandonliu12/nfl-qb-prediction/blob/main/Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries, read files, and clean it further

In [31]:
import pandas as pd
path = 'NFL Play by Play 2009-2018 (v5).csv'
df = pd.read_csv(path)

df['game_date'] = pd.to_datetime(df['game_date'])
df['year'] = df['game_date'].dt.year
df_2018 = df[df['game_date'].dt.year == 2018]
df = df[df['year'] != 2018]

df = df.dropna(subset=['air_yards'])
df_2018 = df_2018.dropna(subset=['air_yards'])



<ipython-input-31-94cae4cd6f99>:3: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [32]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['passer_player_name_encoded'] = encoder.fit_transform(df['passer_player_name'])

In [33]:
import numpy as np
relevant_columns = ['yardline_100','game_seconds_remaining','qtr', 'down', 'air_yards', 'pass_attempt',
                     'complete_pass', 'ep', 'air_epa', 'passer_player_name_encoded', 'pass_length', 'pass_location', 'score_differential_post']

# Select the data using the relevant columns
df_selected = df[relevant_columns]

# Get list of categorical columns
categorical_cols = df_selected.select_dtypes(include=['object']).columns

# Create dummy variables for all categorical columns
df_dummies = pd.get_dummies(df_selected[categorical_cols])

# Combine original numerical data with dummy variables
df_combined = pd.concat([df_selected.select_dtypes(include=[np.number]), df_dummies], axis=1)

df_combined = df_combined.fillna(0)

df_combined.head()

,yardline_100,game_seconds_remaining,qtr,down,air_yards,pass_attempt,complete_pass,ep,air_epa,passer_player_name_encoded,score_differential_post,pass_length_deep,pass_length_short,pass_location_left,pass_location_middle,pass_location_right
1,58.0,3593.0,1,1.0,-3.0,1.0,1.0,2.338000,-0.938735,46,0.0,0,1,1,0,0
3,56.0,3515.0,1,3.0,34.0,1.0,0.0,1.013147,3.412572,46,0.0,1,0,0,0,1
6,98.0,3460.0,1,2.0,3.0,1.0,1.0,-2.094662,-0.521641,180,0.0,0,1,1,0,0
9,43.0,3384.0,1,1.0,-2.0,1.0,1.0,3.414601,-0.680790,46,0.0,0,1,0,0,1
10,40.0,3348.0,1,2.0,-1.0,1.0,1.0,3.199308,-0.877942,46,0.0,0,1,1,0,0


In [34]:
df_2018['passer_player_name_encoded'] = encoder.fit_transform(df_2018['passer_player_name'])

df_selected_2018 = df_2018[relevant_columns]

# Get list of categorical columns
categorical_cols = df_selected_2018.select_dtypes(include=['object']).columns

# Create dummy variables for all categorical columns in 2018 dataframe
df_dummies_2018 = pd.get_dummies(df_selected_2018[categorical_cols])

# Combine original numerical data with dummy variables for 2018 dataframe
df_combined_2018 = pd.concat([df_selected_2018.select_dtypes(include=[np.number]), df_dummies_2018], axis=1)

# Fill NaN values with 0 for 2018 dataframe
df_combined_2018 = df_combined_2018.fillna(0)


Machine Learning Model

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Define target variable
target = df_combined['air_yards']

# Define features
features = df_combined.drop('air_yards', axis=1)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [36]:
from sklearn.ensemble import RandomForestRegressor

# Create a random forest regressor object
random_forest = RandomForestRegressor(random_state=42, n_estimators=100)

# Train the model using the training sets
random_forest.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = random_forest.predict(X_test)


In [37]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(y_test, y_pred))

print(f"Model RMSE: {rmse}")


Model RMSE: 1.8192809211778345


In [38]:
mean_air_yards = df['air_yards'].mean()
mean_air_yards

8.345890389457095

Predict 2018 performance

In [39]:
# Separate the features from the target variable in df_2018
features_2018 = df_combined_2018.drop('air_yards', axis=1)

# Use the trained model to predict 'air_yards' for df_2018
predictions_2018 = random_forest.predict(features_2018)

# Add the predictions as a new column in df_2018
df_2018['predicted_air_yards'] = predictions_2018

In [40]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Separate the actual 'air_yards' values
actuals_2018 = df_2018['air_yards']

# Calculate MAE, MSE, RMSE, and R-squared
mae = mean_absolute_error(actuals_2018, predictions_2018)
mse = mean_squared_error(actuals_2018, predictions_2018)
rmse = np.sqrt(mse)  # or directly use mean_squared_error(actuals_2018, predictions_2018, squared=False)
r2 = r2_score(actuals_2018, predictions_2018)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")


MAE: 0.9069725161133406
MSE: 3.0328396753009854
RMSE: 1.7415050029503174
R-squared: 0.9703736693442978


In [42]:
predicted_df = df_2018[['air_yards', 'predicted_air_yards']]
predicted_df.head()

,air_yards,predicted_air_yards
407337,8.0,6.08
407340,4.0,4.00
407341,-3.0,-3.11
407342,24.0,23.76
407345,1.0,1.00


Continue predicting additional statistic such as pass location and distance

In [ ]:
# Define target variable
target = df_combined['air_yards']

# Define features
features = df_combined.drop('air_yards', axis=1)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create a random forest regressor object
random_forest = RandomForestRegressor(random_state=42, n_estimators=100)

# Train the model using the training sets
random_forest.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = random_forest.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred))

print(f"Model RMSE: {rmse}")

mean_air_yards = df['air_yards'].mean()
mean_air_yards

In [ ]:
# Separate the features from the target variable in df_2018
features_2018 = df_combined_2018.drop('air_yards', axis=1)

# Use the trained model to predict 'air_yards' for df_2018
predictions_2018 = random_forest.predict(features_2018)

# Add the predictions as a new column in df_2018
df_2018['predicted_air_yards'] = predictions_2018

# Separate the actual 'air_yards' values
actuals_2018 = df_2018['air_yards']

# Calculate MAE, MSE, RMSE, and R-squared
mae = mean_absolute_error(actuals_2018, predictions_2018)
mse = mean_squared_error(actuals_2018, predictions_2018)
rmse = np.sqrt(mse)  # or directly use mean_squared_error(actuals_2018, predictions_2018, squared=False)
r2 = r2_score(actuals_2018, predictions_2018)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")
